# **COMMITS_FREQUENCY**

This notebook the creation of the table `COMMITS_FREQUENCY`, that contains the frequency of commits by commiter, that is, the mean number of days between its commits.

First, we import the libraries we need and, then, we read the corresponding csv.


In [1]:
import pandas as pd
import numpy as np

In [2]:
gitCommits = pd.read_csv("../../../data/interim/DataPreparation/CleanData/GIT_COMMITS_clean.csv")
print(gitCommits.shape)
gitCommits

(140653, 7)


,Unnamed: 0,Unnamed: 0.1,projectID,commitHash,author,committer,committerDate
0,0,0,accumulo,e0880e263e4bf8662ba3848405200473a25dfc9f,Keith Turner,Keith Turner,2011-10-04T00:46:07Z
1,1,1,accumulo,e8774c5ec3a35e042f320540b5f7e66ebd2d9e87,Billie Rinaldi,Billie Rinaldi,2011-10-04T16:57:13Z
2,2,2,accumulo,2032ebbd0ed90734da39ca238bbd10dee24d0030,Keith Turner,Keith Turner,2011-10-04T18:39:18Z
3,3,3,accumulo,de297d4932e08625a5df146f0802041bb5aeb892,Billie Rinaldi,Billie Rinaldi,2011-10-04T19:31:01Z
4,4,4,accumulo,34efaae87639a83b60fdb7274de4b45051025a3a,Billie Rinaldi,Billie Rinaldi,2011-10-05T17:19:06Z
...,...,...,...,...,...,...,...
140648,140682,140682,zookeeper,cc900a3b05bc31a237753680c8b00dc5866df4b2,Brian Nixon,Norbert Kalmar,2019-07-15T14:15:03Z
140649,140683,140683,zookeeper,1c83846615701e88749690f06993a6e77452b83c,Ivan Yurchenko,Andor Molnar,2019-07-15T14:46:48Z
140650,140684,140684,zookeeper,f873dcf10e222e220732ab27cc6fc8c0ff0beec6,Andor Molnar,Norbert Kalmar,2019-07-16T09:21:14Z
140651,140685,140685,zookeeper,a6c36b69cc72d7d67e392dab5360007d6f737bef,maoling,Andor Molnar,2019-07-17T13:42:32Z


First of all, we sort the attributes and we calculate the difference between commits of the same committer in seconds.

In [3]:
gitCommits = gitCommits[['committer', 'committerDate']]
newDFsorted = gitCommits.sort_values(by=['committer', 'committerDate']).reset_index()[['committer', 'committerDate']]

In [4]:
newDFsortedCopy = []
committer = newDFsorted.iloc[0,0]
for index, row in newDFsorted.iterrows():
  if index != 0:
    if committer == newDFsorted.iloc[index,0]:
      r = (pd.to_datetime(newDFsorted.iloc[index,1])-pd.to_datetime(newDFsorted.iloc[index-1,1]))
      newDFsortedCopy.append([committer, r])
    else:
      committer = newDFsorted.iloc[index,0]

In [5]:
time_between_commits = pd.DataFrame(newDFsortedCopy)
time_between_commits[1] = time_between_commits[1].dt.total_seconds()
time_between_commits

,0,1
0,-l,92142.0
1,-l,153205.0
2,-l,268.0
3,-l,370.0
4,-l,110.0
...,...,...
139632,Łukasz Gajowy,83849.0
139633,Łukasz Gajowy,1114864.0
139634,Łukasz Gajowy,10654.0
139635,Łukasz Gajowy,80638.0


Then, we calculate the average of the values obtained previously grouping by `committer`.

In [6]:
time_between_commits_commiter = time_between_commits.groupby([0]).mean()
time_between_commits_commiter = pd.DataFrame(time_between_commits_commiter).rename(columns={0:'committer', 1:'time_between_commits'})
time_between_commits_commiter

,time_between_commits
0,
-l,1.629185e+05
1028332163,5.995308e+03
A. J. David Bosschaert,4.035664e+05
A744013,8.799250e+04
Aaron Dossett,9.934671e+04
...,...
yanzhi,9.558100e+04
yoshiki.obata,6.170935e+06
zmanji@apache.org,9.164237e+05


We save this table in a new csv.

In [7]:
time_between_commits_commiter.to_csv('../../../data/interim/DataPreparation/ConstructData/COMMITS_FREQUENCY.csv', header=True)